# Hulu Recommendation System Notebook

## Overview

The aim of this project is to build a model that provides **top 5 movie recommendations** to a user, based on their **ratings of other movies** from Hulu. Our audience in this case is the **Hulu Technology Team** headed by the *CTO* of Hulu. They are holding a competition to upgrade their current machine learning algorithm that oversees the movie recommendations given to a Hulu subscriber. 

## Business/Industry Understanding 

Todays streaming landscape is vast and diverse, with over *1 billion users worldwide* and generating over *500 billion dollars* in revenue.  Hulu currently holds *11% of the market share* within the streaming industry, trailing behind *Netflix* with *25% of the market*, *Amazon Prime Video* with *18%*, *HBO Max* with *17%* and *Disney Plus* *with 14%*.  It's also important to note that *85%* of Hulu users are also subscribed to Netflix.  Currently, Hulu uses a *item-based collaborative filtering algorithm*, this is a successful approach commonly used by many recommender systems. This type of recommender system takes into consideration the ratings given to movies and shows by users as the sole source of information for learning to make recommendations. 

## Data Understanding

Over half of the movies were rated 3 or less times which would skew the movies ratings

## Imports

`Surprise` was the main library we used to produce our *collaborative filtering model*.This library has endogenous methods of splitting our data set, cross validating, performing grid searches as well various different algorithms that can be used to build recommendation systems. 

As you go through this notebook, you will see that we had to use a combination of both surprise and `scikit-learn` methods to ensure the successful deployment of model. Since the syntax of `surprise` and `sklearn` are similar, this made this process much more efficient. This also ensures that the notebook would be readable to anyone with a background in *data science*. We mainly utilized `surprise` to transform the data set to a suitable data format that can be used in *recommendation systems*. We also employed `pickling` to make loading onto any new computer an easier process. 

In [7]:
import pandas as pd

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate,GridSearchCV
from surprise.prediction_algorithms import KNNBasic,SVD, SVDpp, NMF, BaselineOnly, NormalPredictor
from sklearn.model_selection import train_test_split

## EDA

Our Exploratory Data Analysis also includes *data cleaning* and *reformatting*.

In [8]:
movies = pd.read_csv('../../ml-latest-small/movies.csv')
rating = pd.read_csv('../../ml-latest-small/ratings.csv')

We loaded the data in from the two csv files: **movies.csv** and **ratings.csv** 

In [9]:
#looking at the movies df
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


The movies data set has **9742 rows** and **3 columns**, where each row represents **one movie**. The first column : `movieId` corresponds to a **unique id** that defines each movie in this dataset. The second column : `title`  contains the movie title which, in this case, is a combination of both the title of the movie and the year it was released. The final column is `genres` and it contains the `genres` of the movie, some movies are tied to multiple genres. For example, the movie *Toy Story* is considered a Children's movie, but also an action movie, as well as adventure, comedy and fantasy.

In [10]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [11]:
movies.title.value_counts()

Emma (1996)                               2
Eros (2004)                               2
War of the Worlds (2005)                  2
Confessions of a Dangerous Mind (2002)    2
Saturn 3 (1980)                           2
                                         ..
Coco (2017)                               1
Knowing (2009)                            1
Banana Joe (1981)                         1
Four Musketeers, The (1974)               1
Black Mask (Hak hap) (1996)               1
Name: title, Length: 9737, dtype: int64

In [12]:
movies.title.duplicated().sum()

5

In [13]:
#Drop the 5 duplicated movie titles
movies.drop_duplicates(subset='title', inplace=True)

In [14]:
#Sanity check to ensure all duplicates were dropped from title column

movies.title.duplicated().sum()

0

In [15]:
movies[movies.title.duplicated() == True]

,movieId,title,genres


First, we established that there were **no nulls** in the data frame. Then we checked to see whether there were any **duplicates** by looking at the `value_counts` specific to *movie titles* and we noticed **5 duplicated data points**, so we dropped the duplicates present in the data. We then reconfirmed that our data had no duplicates and the size of new data frame without duplicates was **9737**, **5 less** than the original **9742**.

In [16]:
len(movies)

9737

In [17]:
# Split title column into two new columns: Title and year 

movies['Title'] = movies['title'].str.split('(', n=1, expand=True)[0]

movies['year_released'] = movies['title'].str.split('(', n=1, expand=True)[1]

We decided to format the `title` column, so we split it by the `delimiter '()'` so that we could separate the `title` and `year_released`. We felt that we wanted to isolate these two variables as we wanted to see what effect `year_released` would have on the model.  

In [18]:
# Drop original column title

movies.drop(columns='title', inplace=True)

In [19]:
# Sanity Check 
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995)
1,2,Adventure|Children|Fantasy,Jumanji,1995)
2,3,Comedy|Romance,Grumpier Old Men,1995)
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995)
4,5,Comedy,Father of the Bride Part II,1995)
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017)
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017)
9739,193585,Drama,Flint,2017)
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018)


In [20]:
# Format year_released column

movies['year_released'] = movies.year_released.str.replace(')', '')

In [21]:
# Sanity Check to ensure formatting was completed
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017
9739,193585,Drama,Flint,2017
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018


When we originally ran the formatting code, it didn't delete the *close* parenthesis, so we had to write additional code above to fix that and ensure clean data. 

In [22]:
#checking rating df
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


The ratings data frame has **100,836 rows** and **4 columns**, where each row is one rating given by a specific user to a specific movie. The first column : `userId` corresponds to a `unique id` that identifies the **specific user** that rated the **specific movie**, denoted by the second column: `movieId`. This `movieId` is the same identifying key that is present in the `movies` data frame. The third column represents the `rating` that a user gave the movie, on a **1-5 rating scale**. Lastly, the final column corresponds to the `date and time` the rating was made, in the format of a `UNIX timestamp`.

In [23]:
#converted timestamp to datetime
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')

We converted the `UNIX timestamp` into a `date time object`, to determine if we could incorporate the element of time as a feature within our moldel.  However, after reviewing the data, we decided that these datasets will change over time, and are not appropriate for reporting research results. 

In [24]:
rating.rating.value_counts(normalize=True)

4.0    0.265957
3.0    0.198808
5.0    0.131015
3.5    0.130271
4.5    0.084801
2.0    0.074884
2.5    0.055040
1.0    0.027877
1.5    0.017762
0.5    0.013586
Name: rating, dtype: float64

In [25]:
rating.duplicated().sum()

0

In [26]:
rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

First, we established that there were **no nulls** in the data frame, then we checked to see whether there were any **duplicates**, so we did a `value_counts` specific to ratings and confirmed there are **no duplicates**.

In [27]:
# merging rating and movies
movie_rating = movies.merge(rating, on='movieId', how='outer')

In [28]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100849,64997,NaN,NaN,NaN,68.0,2.5,2008-12-28 20:55:15
100850,144606,NaN,NaN,NaN,111.0,4.0,2018-01-31 23:27:37
100851,147002,NaN,NaN,NaN,318.0,4.0,2017-08-08 15:45:52
100852,26958,NaN,NaN,NaN,509.0,3.5,2015-07-04 17:42:33


In [29]:
movie_rating.dropna(inplace=True)

In [30]:
movie_rating['genres']=[row.strip().lower().replace('|',',') for row in movie_rating['genres']]

The code above is to change the genres from its current format into a list that we can iterate over in our final function

In [31]:
movie_rating['userId'].nunique()

610

The code above was to determine how many *unique users* contributed to the *total ratings*, in this case, **610 unique users** made **100854 ratings** of **9737 movies**. 

In [32]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100843,193581,"action,animation,comedy,fantasy",Black Butler: Book of the Atlantic,2017,184.0,4.0,2018-09-16 14:44:42
100844,193583,"animation,comedy,fantasy",No Game No Life: Zero,2017,184.0,3.5,2018-09-16 14:52:25
100845,193585,drama,Flint,2017,184.0,3.5,2018-09-16 14:56:45
100846,193587,"action,animation",Bungo Stray Dogs: Dead Apple,2018,184.0,3.5,2018-09-16 15:00:21


# Splits

The `train test split` process was a little bit different than usual. This is because of how the `surprise` library's methods for `cross validation` and `GridSearch` are comprised. A different input type is needed for `GridSearch` and `cross validation` methods compared to the `.fit` and `.test` methods. The documentation recommends doing `cross validation` **OR** the `train test split`. Since we wanted to do both we had to `train test split` **twice:** `train_data` and `test_data`, defined below are used for `fitting` and `cross validation` while `train_data2` is used for predictions. 

In [33]:
#for function building/final model
reader = Reader(rating_scale=(1, 5))
complete_data = Dataset.load_from_df(movie_rating[['userId', 'movieId', 'rating']], reader)

In [34]:
#use sklearn for model selection/choosing 
train, test = train_test_split(movie_rating, random_state=42)

reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
test_data = Dataset.load_from_df(test[['userId', 'movieId', 'rating']], reader)


train_data2 = train_data.build_full_trainset()
test_data2 = test_data.build_full_trainset().build_testset()

# Modeling

### Baseline/dummy

The first step in our **iterative modeling process** is to create and test a `dummy model`, which will act as our `base line RSME (root mean squared error) score` that we compare to when judging the **validity** and **performance** of our future models.  

In [35]:
from surprise.prediction_algorithms.random_pred import NormalPredictor

In [36]:
dummy_model = NormalPredictor()
cross_validate(dummy_model, train_data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4311  1.4196  1.4181  1.4035  1.4193  1.4183  0.0088  
Fit time          0.06    0.06    0.06    0.06    0.06    0.06    0.00    
Test time         0.07    0.07    0.11    0.06    0.11    0.08    0.02    


{'test_rmse': array([1.43105067, 1.41959322, 1.41814913, 1.40350055, 1.41927847]),
 'fit_time': (0.0620269775390625,
  0.06451296806335449,
  0.06005096435546875,
  0.06082296371459961,
  0.06258106231689453),
 'test_time': (0.06969332695007324,
  0.0678257942199707,
  0.10949277877807617,
  0.0640878677368164,
  0.11128902435302734)}

In [37]:
predictions = dummy_model.fit(train_data2).test(test_data2)
baseline_rsme = accuracy.rmse(predictions)
baseline_rsme

RMSE: 1.4252


1.4252294311652969

An RSME score of **1.425** is considered to be not good.

### KNNBasics

The `k-nearest neighbors` (KNN) algorithm is a *simple*, *supervised machine learning* algorithm that can be used to solve both `classification` and `regression` problems. In this case, we are using it as the basis for our `recommendation system`. The premise is that the rating of movies *unseen* by the user will be **predicted** using *ratings of similar users*. In this case, the nearest "neighbors".

In [38]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.6s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.1s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   29.5s finished


In [39]:
base_model.best_params

{'rmse': {'k': 10, 'min_k': 5}, 'mae': {'k': 10, 'min_k': 5}}

In [42]:
#trying different parameters
param_grid = {'k':[5, 10, 15],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.8s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   22.1s finished


In [43]:
base_model.best_params

{'rmse': {'k': 15, 'min_k': 5}, 'mae': {'k': 15, 'min_k': 5}}

In [44]:
param_grid = {'k':[10, 15, 20],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.7s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.3s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   26.9s finished


In [45]:
base_model.best_params

{'rmse': {'k': 15, 'min_k': 5}, 'mae': {'k': 15, 'min_k': 5}}

We performed a `GridSearch` until the `hyper-parameters` were tuned to what is considered the *optimal amount*.

#### KNN Cross Validation

In [49]:
#instantiate KNN model 
model1=KNNBasic(k=15, min_k=5)

In [50]:
#instaniatite and perform cross val with training data
knn_cv_results = cross_validate(model1, train_data, measures=['RMSE'], cv=5, verbose=True)
knn_cv_results

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9564  0.9564  0.9584  0.9447  0.9469  0.9526  0.0056  
Fit time          0.03    0.04    0.04    0.04    0.04    0.04    0.00    
Test time         0.49    0.47    0.48    0.49    0.50    0.49    0.01    


{'test_rmse': array([0.9563699 , 0.95635181, 0.95841547, 0.94470901, 0.94693403]),
 'fit_time': (0.03301811218261719,
  0.03883504867553711,
  0.04093122482299805,
  0.03986310958862305,
  0.04034996032714844),
 'test_time': (0.4919579029083252,
  0.46941518783569336,
  0.47850966453552246,
  0.4915149211883545,
  0.5024828910827637)}

In [51]:
#use model to predict
model1.predict(uid=10, iid = 20)
#this model just predicted what user 10 would predict movie 20

Prediction(uid=10, iid=20, r_ui=None, est=3.1376577645284103, details={'actual_k': 7, 'was_impossible': False})

In [52]:
knn_rsme = knn_cv_results['test_rmse'].mean()
knn_rsme

0.9525560430621016

Looking at the output above we can determine that our model predicted that **user 10** would rate **movie 20** a **2.5** give or take our **RSME** which is **.95**.

### SVD

After defining our `baseline model` and exploring the `KNN` algorithm, we decided to try using `Singular Value Decomposition (SVD)` algorithm, in the hopes that it would improve the `accuracy` of our model and lower the `RSME`. The `surprise` library uses a form of `SVD` called `Funk's SVD`, the premise behind it is a **matrix factorization-based model**. This means it *transforms* a **user-item matrix** (in our case, a *user-movie ratings matrix*)into latent factor matrices. It is also important to note that errors are *minimized* using `stochastic gradient descent`.  This means that both `regularization` and `learning rate` are `hyper-parameters` that we can tune. Number of `factors` and number of `epochs` are also `hyper-parameters` we tuned.

In [54]:
model2 = SVD()

In [55]:
cross_validate(model2, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8792  0.8832  0.8825  0.8815  0.8781  0.8809  0.0019  
MAE (testset)     0.6778  0.6820  0.6802  0.6769  0.6734  0.6781  0.0029  
Fit time          3.24    3.18    3.21    3.20    3.16    3.20    0.03    
Test time         0.07    0.07    0.07    0.07    0.14    0.08    0.03    


{'test_rmse': array([0.87920985, 0.88315022, 0.88245983, 0.88152088, 0.87807586]),
 'test_mae': array([0.67781055, 0.68199604, 0.6801517 , 0.67693074, 0.6733749 ]),
 'fit_time': (3.2427380084991455,
  3.184597969055176,
  3.2120449542999268,
  3.201364755630493,
  3.1575000286102295),
 'test_time': (0.06750988960266113,
  0.07180213928222656,
  0.0703279972076416,
  0.06795191764831543,
  0.1373140811920166)}

In [56]:
param_grid = {'n_factors':[10,20,50],'n_epochs': [5, 10, 15], 'lr_all': [0.002,0.005,0.01],
             'reg_all': [0.2,0.4,0.6]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:  5.2min finished


In [57]:
gs_model2.best_params

{'rmse': {'n_factors': 20, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2},
 'mae': {'n_factors': 50, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}}

In [58]:
param_grid = {'n_factors':[25, 50, 75],'n_epochs': [15, 30, 45], 'lr_all': [0.01,0.05,0.1],
             'reg_all': [0.1,0.2,0.3]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed: 19.9min finished


In [59]:
gs_model2.best_params

{'rmse': {'n_factors': 75, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1},
 'mae': {'n_factors': 75, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1}}

In [60]:

cross_validate(SVD(n_factors=75,n_epochs=45,lr_all=0.01,reg_all=0.1), train_data, measures=['RMSE', 'MAE'], 
               cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8616  0.8582  0.8638  0.8620  0.8657  0.8622  0.0025  
MAE (testset)     0.6606  0.6615  0.6644  0.6630  0.6595  0.6618  0.0017  
Fit time          5.78    5.91    5.79    5.73    5.76    5.80    0.06    
Test time         0.13    0.07    0.07    0.07    0.07    0.08    0.03    


{'test_rmse': array([0.86156596, 0.85818586, 0.86376428, 0.86197783, 0.86568174]),
 'test_mae': array([0.66064796, 0.66147984, 0.66437366, 0.66296262, 0.65949221]),
 'fit_time': (5.783094882965088,
  5.908195972442627,
  5.791035175323486,
  5.733952045440674,
  5.761847019195557),
 'test_time': (0.13464617729187012,
  0.06612586975097656,
  0.06586599349975586,
  0.06646418571472168,
  0.06558489799499512)}

In [ ]:
param_grid = {'n_factors':[75, 100, 125],'n_epochs': [45, 60, 75], 'lr_all': [.005, 0.01, .015],
             'reg_all': [.05, 0.1, 0.15]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   11.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   17.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   23.3s remaining:    0.0s


In [ ]:
gs_model2.best_params

In [ ]:
svd_cv_results = cross_validate(SVD(n_factors=125,n_epochs=45,lr_all=0.015,reg_all=0.1), train_data, measures=['RMSE'], 
               cv=5, verbose=True)
svd_cv_results

In [ ]:
svd_rsme = svd_cv_results['test_rmse'].mean()
svd_rsme

### NMF

The **final algorithm** we explored in the `surprise` Library was `Non-negative Matrix Factorization (NMF)`. The only difference between this algorithm and the previous algorithm `SVD` is that in `NMF`, a specific *step size* is set for the `stochastic gradient descent` process of `regularization`. This is done to ensure that all **user** and **item factors** are kept positive. While this model can be prone to **over-fitting**, this can be mitigated with steps to **reduce the dimensionality** of our **data/factors**. 

In [ ]:
model3 = NMF()

In [ ]:
cross_validate(model3, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
param_grid = {'n_factors':[10,15,20],'n_epochs': [25, 50, 75]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

In [ ]:
gs_model3.best_params

In [ ]:
cross_validate(NMF(n_factors=10,n_epochs=25), train_data, measures=['RMSE', 'MAE'], 
               cv=5, verbose=True,  n_jobs= -2)

In [ ]:
param_grid = {'n_factors':[3,5,10],'n_epochs': [15, 20, 25]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

In [ ]:
gs_model3.best_params

In [ ]:
nmf_cv_results = cross_validate(NMF(n_factors=20,n_epochs=50), train_data, measures=['RMSE'], 
               cv=5, verbose=True)

In [ ]:
nmf_rsme = nmf_cv_results['test_rmse'].mean()
nmf_rsme

### Final Model Evaluation

When we compare the **best model** with *each different* algorithm explored, with `hyper-parameters` tuned to *optimize*, we found that our **best model** using `SVD` is the model which exhibits the *lowest* `RSME` in `cross validation`. Therefore that is the model we have determined to be our **final model**.

In [ ]:
baseline_rsme

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = ('Baseline', 'KNN', 'NMF', 'SVD')
y_pos = np.arange(len(objects))
performance = [baseline_rsme, knn_rsme, nmf_rsme, svd_rsme]

plt.bar(y_pos, performance, align='center', alpha=0.5, color=['red', 'blue', 'blue', 'green'])
plt.xticks(y_pos, objects)
plt.ylabel('RMSE Accuracy')
plt.title('Model Performance')

plt.show()

In [ ]:
final_model = SVD(n_factors=125,n_epochs=45,lr_all=0.015,reg_all=0.1)

In [ ]:
predictions = final_model.fit(train_data2).test(test_data2)

In [ ]:
accuracy.rmse(predictions)

## Deployment

In terms of deploying our final model, we used retrained our final model on the complete data set. To fit the complete data set, we transformed it into a surprise train set. 

In [ ]:
#create trainset object from surprise dataset
full_data = complete_data.build_full_trainset()

In [ ]:
final_model.fit(full_data)

#### Function Building

Below is the process in which we built our deployment function. The purpose of this function is to take 3 inputs from the reader and give movie recommendations based on those inputs

In [ ]:
#setting index to ensure dropping
userdf = movie_rating.set_index('userId')

First we reset the index in our merged data frame (*movie_rating*)  to have *userId* as the index. This is to ensure the matrix that is inputted into our recommender function is in the correct format.

In [ ]:
userdf

In [ ]:
#create 
userdf1 = userdf['movieId']

Here, we are trying to isolate the movie Id column, so we can use it later in the function as a new 

In [ ]:
userdf1.loc[9,]

In [ ]:
user_rate = list(userdf1.loc[9,])

This is a sanity check, we're trying to create a list of movies that User 9 has seen. The purpose of this is to take these movies out of the data set that is being predicted on. This is because we don't want to recommended movies to a user that they have already seen/rated. 

In [ ]:
user_rate

In [ ]:
#setting index so we can drop base off of specific movieId
moviedf = movies.set_index('movieId')

In [ ]:
moviedf

In [ ]:
moviedf.drop(user_rate, inplace = True)

In [ ]:
moviedf

In [ ]:
#reset index to be able to use .apply on the movieId
moviedf = moviedf.reset_index()

In [ ]:
moviedf

In [ ]:
moviedf['est_rating'] = moviedf['movieId'].apply(lambda x: final_model.predict(9, x).est)
moviedf.sort_values(by='est_rating', ascending=False, inplace=True)

In [ ]:
moviedf.head(5)

#### Function

In [ ]:
#creating a variable that only has the userid and movies that they've rated
#this uses the original train dataset not surprise's version
users_movies_seen = movie_rating[['movieId', 'userId']]
#setting user id as index to make it easier to use loc on it
users_movies_seen = users_movies_seen.set_index('userId')

In [ ]:
def recommender():
    user = float(input('userId: '))
    genre = input('What genres are you interested in? ')
    num_recs = int(input('How many recomendations would you like? '))

    #using the train data
    #create the seen_movie
    seen_movie = list(users_movies_seen.loc[user, 'movieId'])

    #create df with all the unseen/unrated movies
    train_copy = movies.copy()
    train_copy2 = train_copy.set_index('movieId')

    #using the not_seen df, create a rating for a user
    not_seen = train_copy2.drop(index = seen_movie)

    #subsetting the df to unseen movies of a certain genre
    not_seen = not_seen[not_seen['genres'].str.contains(genre)]
    
    #reset not_seen index
    not_seen = not_seen.reset_index()
    
    #creating an est rating and the sorted it
    not_seen['est_rating'] = not_seen['movieId'].apply(lambda x: final_model.predict(user, x).est)
    not_seen.sort_values(by='est_rating', ascending=False, inplace=True)

    #returns the recommendations that the model predicts to be the highest rating
    return (not_seen.head(num_recs))

In [ ]:
movies

In [ ]:
recommender()

## Recommendations 

Based on the performance of our model we can recommend it be used in the following ways: 
 1. Use our model to predict **rating of movies** that the user hasn't rated by looking at the **ratings** for those movies done by **similar users**.
 2. Use these **predicted ratings** as the basis for providing **recommendations** to users using the **app version** of our deployed recommendation system.
 3. Use the **app version** to filter based recommendations based on `genre`, including the movies that have *more than one* `genre` tied to it. 
 4. Use this as the basis to increase interest in films vs TV shows 

## Next Steps

1. Use more current data (our data set is from the 1996-2018), since tastes in movies and TV shows change so much over time, this would be a crucial next step in developing our improving our recommendation system model performance.

2. Use more first party data ( or dataset is from MovieLens), by using data native to Hulu we can gain a better understanding of the preferences of Hulu-specific users as opposed to users in general.

3. Use more ratings, we chose to use the smaller dataset in our analysis because of time constraints. We could have chosen to use the bigger set however we decided not to. This analysis should be re-done with more data.

4. Incorporate a content based filtering algorithm to address cold start problem 

5. We would do A/B testing on the res system and subscriber retention

6. Build the second version of our app

